## Notebook Imports

In [1]:
import pandas as pd
import numpy as np

## Constants

In [2]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'
VOCAB_SIZE = 2500

## Read and Load Features from .txt Files into NumPy Array

In [4]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [6]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [7]:
sparse_train_data[-5:]

array([[5795, 2056,    0,    1],
       [5795, 2059,    0,    1],
       [5795, 2284,    0,    1],
       [5795, 2359,    0,    1],
       [5795, 2442,    0,    1]])

In [8]:
print('No of rows in training file', sparse_train_data.shape[0])
print('No of rows in test file', sparse_test_data.shape[0])

No of rows in training file 258328
No of rows in test file 117640


In [10]:
print('No of emails in training file', np.unique(sparse_train_data[:, 0]).size)

No of emails in training file 4015


In [11]:
print('No of emails in test file', np.unique(sparse_test_data[:, 0]).size)

No of emails in test file 1724
